## **Phase 2**

In [1]:
!python -m pip install --upgrade firebase-admin pandas pyarrow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 913.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd


In [4]:
cred = credentials.Certificate("/content/drive/My Drive/firebase.json")
firebase_admin.initialize_app(cred)

db = firestore.client()


In [18]:
df = pd.read_csv("/content/drive/My Drive/song1 database.csv")
df.head() #fun print 5 rows by defult




,track_ID,artist_name,genres,followers,artist_popularity,artist_url,track_name,album_name,release_date,duration_ms,explicit,track_popularity,energy,mode
0,0,Ariana Grande,pop,98934105,89,https://open.spotify.com/artist/66CXWjxzNUsdJx...,we can't be friends (wait for your love),eternal sunshine,3/8/2024,228639,False,89,0.646,1
1,1,Ariana Grande,pop,98934105,85,https://open.spotify.com/artist/66CXWjxzNUsdJx...,the boy is mine,eternal sunshine,3/8/2024,173639,True,85,0.630,0
2,2,Ariana Grande,pop,98934105,83,https://open.spotify.com/artist/66CXWjxzNUsdJx...,intro (end of the world),eternal sunshine,3/8/2024,92400,True,83,0.362,1
3,3,Ariana Grande,pop,98934105,80,https://open.spotify.com/artist/66CXWjxzNUsdJx...,Save Your Tears (Remix) (with Ariana Grande) -...,After Hours (Deluxe),3/20/2020,191013,False,80,0.825,1
4,4,Ariana Grande,pop,98934105,79,https://open.spotify.com/artist/66CXWjxzNUsdJx...,"yes, and?",eternal sunshine,3/8/2024,214994,True,79,0.775,1


In [36]:

df = pd.read_csv("/content/drive/My Drive/song1 database.csv")

for idx, row in df.iterrows():
    doc = {
        "track_id": row.get("track_ID"),
        "track_name": row.get("track_name"),
        "artist": {
            "name": row.get("artist_name"),
            "popularity": row.get("artist_popularity"),
            "followers": row.get("followers"),
            "genres": row.get("genres"),
            "url": row.get("artist_url")
        },
        "album": row.get("album_name"),
        "release_date": row.get("release_date"),
        "duration_ms": row.get("duration_ms"),
        "explicit": row.get("explicit"),
        "track_popularity": row.get("track_popularity"),
        "audio_features": {
            "energy": row.get("energy"),
            "mode": row.get("mode")
        }
    }
    db.collection("spotify_tracks").add(doc)

print("Inserted data into Firebase Firestore successfully.")



Inserted data into Firebase Firestore successfully.


In [38]:
# get all songs query
songs = db.collection("spotify_tracks").order_by('track_id').stream()
for song in songs:
    print(song.to_dict())

{'track_id': 0, 'track_popularity': 89, 'release_date': '3/8/2024', 'duration_ms': 228639, 'audio_features': {'energy': 0.646, 'mode': 1}, 'track_name': "we can't be friends (wait for your love)", 'album': 'eternal sunshine', 'artist': {'followers': 98934105, 'popularity': 89, 'genres': 'pop', 'url': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR', 'name': 'Ariana Grande'}, 'explicit': False}
{'track_id': 1, 'track_popularity': 85, 'release_date': '3/8/2024', 'duration_ms': 173639, 'audio_features': {'energy': 0.63, 'mode': 0}, 'track_name': 'the boy is mine', 'album': 'eternal sunshine', 'artist': {'followers': 98934105, 'popularity': 85, 'genres': 'pop', 'url': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR', 'name': 'Ariana Grande'}, 'explicit': True}
{'track_id': 2, 'track_popularity': 83, 'release_date': '3/8/2024', 'duration_ms': 92400, 'audio_features': {'energy': 0.362, 'mode': 1}, 'track_name': 'intro (end of the world)', 'album': 'eternal sunshine', 'artist'

In [39]:
# Find songs by artist query
query = db.collection("spotify_tracks").where("artist.name", "==", "Ariana Grande").stream()

for doc in query:
    data = doc.to_dict()
    print(f"{data['track_name']} by {data['artist']['name']} (popularity: {data['artist']['popularity']})")




7 rings by Ariana Grande (popularity: 82)
yes, and? by Ariana Grande (popularity: 79)
Die For You (with Ariana Grande) - Remix by Ariana Grande (popularity: 78)
Dangerous Woman by Ariana Grande (popularity: 70)
intro (end of the world) by Ariana Grande (popularity: 83)
One Last Time by Ariana Grande (popularity: 82)
Save Your Tears (Remix) (with Ariana Grande) - Bonus Track by Ariana Grande (popularity: 80)
the boy is mine by Ariana Grande (popularity: 85)
Into You by Ariana Grande (popularity: 70)
we can't be friends (wait for your love) by Ariana Grande (popularity: 89)


/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:303: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
